从零开始实现softmax回归

In [1]:
import torch
from IPython import display
from d2l import torch as d2l

batch_size=256
train_iter,test_iter=d2l.load_data_fashion_mnist(batch_size)

展平每个图像，将它们视为长度784的向量。因为数据集有10个类别，所以输出维度为10

In [2]:
num_inputs=784
num_outputs=10

W=torch.normal(0,0.01,size=(num_inputs,num_outputs),requires_grad=True) # (权重设置为高斯随机分布的值）均值为0,方差为0.01
b=torch.zeros(num_outputs,requires_grad=True)

给定一个矩阵X，我们可以对所有元素求和

In [3]:
X=torch.tensor([[1.0,2.0,3.0],[4.0,5.0,6.0]])
X.sum(0,keepdim=True),X.sum(1,keepdim=True)#按行, 按列

(tensor([[5., 7., 9.]]),
 tensor([[ 6.],
         [15.]]))

实现softmax

In [4]:
def softmax(X):
    X_exp=torch.exp(X)
    partition=X_exp.sum(1,keepdim=True)
    return X_exp/partition      #这里用了广播机制`

In [5]:
X=torch.normal(0,1,(2,5))
#在 PyTorch 中生成一个形状为 (2, 5) 的张量 X，其中每个元素都是从均值为 0、标准差为 1 的正态分布（高斯分布）中随机抽取的。
X_prob=softmax(X)
X_prob,X_prob.sum(1)

(tensor([[0.0525, 0.3028, 0.2080, 0.2249, 0.2118],
         [0.3593, 0.0717, 0.2108, 0.2130, 0.1451]]),
 tensor([1., 1.]))

In [6]:
def net(X):
    return softmax(torch.matmul(X.reshape((-1,W.shape[0])),W)+b)
    #(-1,W.shape[0])表示把X初始化为[batch_size,W行数）
    #然后和W相乘，得到(batch_size,W列数)的矩阵
    #最后加上b ，b的形状应该是(W列数, 1)

In [7]:
y=torch.tensor([0,2])
y_hat=torch.tensor([[0.1,0.3,0.6],[0.3,0.2,0.5]])
y_hat[[0,1],y] 
#[0,1]取得第0号，第1号元素，分别是0和2，然后把0，2带入y_hat索引取（0，0 

tensor([0.1000, 0.5000])

实现交叉熵损失函数

In [8]:
def cross_entropy(y_hat,y):
    return -torch.log(y_hat[range(len(y_hat)),y])
cross_entropy(y_hat,y)
#输出第一个代表样本0的损失,第二个代表样本1的损失。

tensor([2.3026, 0.6931])

将预测类别与真实y元素进行比较

In [9]:
# accuracy函数用来求出被正确分类的样本数
def accuracy(y_hat,y):
    if len(y_hat.shape)>1 and y_hat.shape[1]>1:
        y_hat=y_hat.argmax(axis=1)
    cmp=y_hat.type(y.dtype)==y #把y_hat类型转成和y一致，再与y进行比较
    return float(cmp.type(y.dtype).sum())

accuracy(y_hat,y)/len(y)

0.5

我们也可以评估在任意模型net的准确率

In [10]:
def evaluate_accuracy(net,data_iter):
    # 计算在指定数据集上模型的精度
    if isinstance(net,torch.nn.Module):
        net.eval()
    metric=Accumulator(2)
    for X,y in data_iter:
        metric.add(accuracy(net(X),y),y.numel())
    return metric[0]/metric[1]

In [11]:
class Accumulator:
    def __init__(self,n):
        self.data=[0.0]*n
        
    def add(self,*args):
        self.data=[a+float(b) for a,b in zip(self.data,args)]
        
    def reset(self):
        self.data=[0.0]*len(self.data)
        
    def __getitem__(self,idx):
        return self.data[idx]
        
evaluate_accuracy(net,test_iter)

0.0975